In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np 
import pandas as pd 
import tensorflow as tf
import sys
import collections
import tokenization #
import json
import logging

In [2]:
train_df = pd.read_csv("./data/train_20200228.csv")
val_df = pd.read_csv("./data/dev_20200228.csv")

In [3]:
train_df.head()

,id,category,query1,query2,label
0,0,咳血,"剧烈运动后咯血,是怎么了?",剧烈运动后咯血是什么原因？,1
1,1,咳血,"剧烈运动后咯血,是怎么了?",剧烈运动后为什么会咯血？,1
2,2,咳血,"剧烈运动后咯血,是怎么了?",剧烈运动后咯血，应该怎么处理？,0
3,3,咳血,"剧烈运动后咯血,是怎么了?",剧烈运动后咯血，需要就医吗？,0
4,4,咳血,"剧烈运动后咯血,是怎么了?",剧烈运动后咯血，是否很严重？,0


In [4]:
val_df.head()

,id,category,query1,query2,label
0,0,咳血,请问呕血与咯血有什么区别？,请问呕血与咯血这两者之间有什么区别？,1
1,1,咳血,请问呕血与咯血有什么区别？,请问呕血与咯血异同？,1
2,2,咳血,请问呕血与咯血有什么区别？,请问呕血与咯血怎么治疗？,0
3,3,咳血,请问呕血与咯血有什么区别？,请问呕血与咯血是什么原因导致的？,0
4,4,咳血,请问呕血与咯血有什么区别？,请问呕血与咯血与其他疾病有关联吗？,0


# 变成bert支持的数据格式
- token_id
- input_mask 
- segment_id

思考下：

query1 和query2 合并成一个句子

[cls] query1 [sep] query2 [sep]

In [5]:
import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file= "./preporcess_data/bert_zh_L-12_H-768_A-12_2/assets/vocab.txt", do_lower_case=True)

In [6]:
a = tokenizer.tokenize("深度之眼")

1、句子切割  句子过长需要截断


2、两个句子整合一个句子[cls] query1 [sep] query2 [sep]

3、句子编码 （segment_ids, input_mask）



In [7]:
tokenizer.convert_tokens_to_ids(a)

[3918, 2428, 722, 4706]

In [8]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self,idx, category, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.id = idx
        self.category = category
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
        
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 input_mask,
                 segment_ids,
                 label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    #query1
    tokens_a = tokenizer.tokenize(example.text_a)
    #query 2
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    #截断
    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)
    # [cls] [query1] [sep] [query2] [sep]
    # 0       0       0     1       1
        
        
    #句子编码
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    label_id = label_map[example.label]
    #类记录一条问题，也可以用字典
    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id=label_id)
    return feature


In [9]:

def file_based_convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.io.TFRecordWriter(output_file) # 

    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature([feature.label_id])


        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


In [10]:

def _create_examples(lines,set_type='train'):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):

        idx = line[0]
        category = tokenization.convert_to_unicode(line[1])
        text_a = tokenization.convert_to_unicode(line[2])
        text_b = tokenization.convert_to_unicode(line[3])
        if set_type == "test":
            label = 0
        else:
            label = line[4]
        examples.append(
            InputExample(idx=idx,category=category, text_a=text_a, text_b=text_b, label=label))
    return examples


In [11]:
Examples = _create_examples(train_df.values,set_type='train')

In [12]:
file_based_convert_examples_to_features(Examples,label_list=[0,1], max_seq_length=40, tokenizer=tokenizer, output_file="./preporcess_data/train.tfrecord")

In [13]:
a = convert_single_example(0, Examples[0], label_list=[0,1], max_seq_length=40, tokenizer=tokenizer)


In [14]:
a.segment_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
seq_length= 40

In [16]:
name_to_features = {
    'input_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
    'input_mask': tf.io.FixedLenFeature([seq_length], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
    'label_ids': tf.io.FixedLenFeature([1], tf.int64),
}
def decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.cast(t, tf.int32)
        example[name] = t

    return {"input_ids":example['input_ids'],"input_mask":example['input_mask'],"segment_ids":example['segment_ids']},example['label_ids']


In [17]:
train_ds = tf.data.TFRecordDataset("./preporcess_data/train.tfrecord")
train_ds = train_ds.map(
    lambda record: decode_record(record, name_to_features),
    num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().batch(32)

In [18]:
for line in train_ds:
    print(line)
    break

({'input_ids': <tf.Tensor: shape=(32, 40), dtype=int32, numpy=
array([[ 101, 1196, 4164, ...,    0,    0,    0],
       [ 101, 1196, 4164, ...,    0,    0,    0],
       [ 101, 1196, 4164, ...,    0,    0,    0],
       ...,
       [ 101, 2769, 1744, ...,    0,    0,    0],
       [ 101, 1920, 1492, ...,    0,    0,    0],
       [ 101, 1920, 1492, ...,    0,    0,    0]])>, 'input_mask': <tf.Tensor: shape=(32, 40), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'segment_ids': <tf.Tensor: shape=(32, 40), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(32, 1), dtype=int32, numpy=
array([[1],
       [1],
       [0],

In [19]:
def classifier_model(num_labels,
                     max_seq_length=None,
                     final_layer_initializer=None,
                     hub_module_url=None,
                     hub_module_trainable=True):
    """BERT classifier model in functional API style.

    Construct a Keras model for predicting `num_labels` outputs from an input with
    maximum sequence length `max_seq_length`.

    Args:
      bert_config: BertConfig or AlbertConfig, the config defines the core BERT or
        ALBERT model.
      num_labels: integer, the number of classes.
      max_seq_length: integer, the maximum input sequence length.
      final_layer_initializer: Initializer for final dense layer. Defaulted
        TruncatedNormal initializer.
      hub_module_url: TF-Hub path/url to Bert module.
      hub_module_trainable: True to finetune layers in the hub module.

    Returns:
      Combined prediction model (words, mask, type) -> (one-hot labels)
      BERT sub-model (words, mask, type) -> (bert_outputs)
    """
    if final_layer_initializer is not None:
        initializer = final_layer_initializer
    else:
        initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02)


    input_word_ids = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name='input_ids')
    input_mask = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.layers.Input(
        shape=(max_seq_length,), dtype=tf.int32, name='segment_ids')
    
    bert_model = hub.KerasLayer(hub_module_url, trainable=hub_module_trainable)  #重点
    
    
    pooled_output, _ = bert_model([input_word_ids, input_mask, input_type_ids]) #[cls] [sequence ] #768
    output = tf.keras.layers.Dropout(rate=0.1)(pooled_output)

    output = tf.keras.layers.Dense(num_labels, kernel_initializer=initializer, name='output')(output)
    return tf.keras.Model(
        inputs={
            'input_ids': input_word_ids,
            'input_mask': input_mask,
            'segment_ids': input_type_ids
        },
        outputs=output), bert_model


In [20]:
import tensorflow_hub as hub
model, core_model = classifier_model(num_labels=2,max_seq_length=40,hub_module_url='./preporcess_data/bert_zh_L-12_H-768_A-12_2',hub_module_trainable=True)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 40)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 102267649   input_ids[0][0]                  
                                                                 input_mask[0][0]             

In [22]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-5),
             metrics=['accuracy'])

In [ ]:
!tensorflow==2.1.0  2.0.0

In [ ]:
model.fit(train_ds,epochs=10,steps_per_epoch=274)
#0.98

Train for 274 steps
Epoch 1/10
107/274 [==========>...................] - ETA: 16:58 - loss: 0.4573 - accuracy: 0.7754

In [ ]:
#有个验证集
#版本问题


In [ ]:
tf.saved_model.save(model,"./save_models/bert_version1")

In [26]:
restored_saved_model = tf.saved_model.load("./save_models/bert_version1")

In [27]:
f = restored_saved_model.signatures["serving_default"]

In [38]:
test_sample = convert_single_example(0, Examples[0], label_list=[0,1], max_seq_length=40, tokenizer=tokenizer)


In [42]:
!saved_model_cli show --dir ./save_models/bert_version1 --all

2020-07-03 16:12:27.666551: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-07-03 16:12:27.666676: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-07-03 16:12:27.666696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef co

In [39]:
input_ids = test_sample.input_ids
input_mask = test_sample.input_mask
segment_ids = test_sample.segment_ids

In [ ]:
input_ids

In [46]:
f(input_ids=tf.constant([input_ids,input_ids]),input_mask=tf.constant([input_mask,input_mask]),segment_ids=tf.constant([segment_ids,segment_ids]) )


{'output': <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[-3.4082842,  2.4501426],
        [-3.4082842,  2.4501426]], dtype=float32)>}